Format Profiling via the Solr API
----------------------------------------------

Having indexed our content using the [webarchive-discovery](https://github.com/ukwa/webarchive-discovery) system, the resulting Solr API exposes lots of aspects of the majority of our collection. Let's use it to understand our format profiles.

First, we need to set up the specific Solr Cloud collection we're going to use. To make it easier to develop the queries we need, we use a smaller collection first (to get quick results), and then re-run against a [collection alias](https://lucene.apache.org/solr/guide/8_1/aliases.html) that includes all indexed material.


In [63]:
# A relatively small collection:
#solr_collection = "selective-20190221"
# A much larger collection:
solr_collection = "all"

# the full endpoint address:
solr_endpoint = "http://solr.api.wa.bl.uk/solr/%s" % solr_collection

To check all the basics are working, we do a simple query to see how many records we have.

In [64]:
import json
import requests

# Prevent web proxies from interfering:
proxies = {
  "http": None,
  "https": None,
}

# Set the query parameters - match everything, return JSON:
q = {
    'q': '*:*',
    'wt': 'json'
}

# Run the request:
r = requests.post(url='%s/select' % solr_endpoint, data=q, proxies=proxies)

# Report request time
print("Request took %s seconds." % r.elapsed.total_seconds())

# Parse the response:
results = json.loads(r.text)

# Print the number of hits:
total_records = results['response']['numFound']
print("Total records: %i" % total_records)

Request took 0.042057 seconds.
Total records: 4782951820


By default, the indexer code runs [Apache Tika](https://tika.apache.org/) and the [Nanite](https://github.com/openpreserve/nanite) version of the [DROID](https://github.com/digital-preservation/droid) format identification engine.  The separate results are all stored, and are combined to produce an estimate for the format of each item.

Formats that we can't match against [PRONOM](http://www.nationalarchives.gov.uk/PRONOM/) get allocated a `content_type_droid` value of `application/octet-stream` as a way of saying 'we tried to identify this item and found no match' (rather than 'the identification process failed', which would be recorded as an empty result).

So, we can see how many items DROID/Nanite failed to identify via this query:

In [65]:
# Set the query parameters - match DROID misses:
q = {
    'q': 'content_type_droid:"application/octet-stream"',
    'wt': 'json'
}

# Run the request:
r = requests.post(url='%s/select' % solr_endpoint, data=q, proxies=proxies)

# Report request time
print("Request took %s seconds." % r.elapsed.total_seconds())

# Parse the response:
results = json.loads(r.text)

# Print the number of hits:
no_pronom_count = results['response']['numFound']
print("Total records that could not be matched to PRONOM records: %i" % no_pronom_count )
print("Percentage of records that could not be matched to PRONOM records: %f" % (100.0 * no_pronom_count / total_records)) 


Request took 0.156468 seconds.
Total records that could not be matched to PRONOM records: 174225516
Percentage of records that could not be matched to PRONOM records: 3.642636


So, only a small percentage is unrecognised, but due to the scale of the collection, this still corresponds to many millions of records.

Knowing the known and unknown unknowns
----------------------------------------------------------------

We can start to break them down by looking at what Apache Tika thought of those DROID 'misses'...

In [83]:
# Set the query parameters - match DROID misses:
q = {
    'q': 'content_type_droid:"application/octet-stream"',
    'wt': 'json',
    'indent': 'true',
    # Don't bother returning any individual results
    'rows': 0,
    # Use the faceting feature to get more details:
    'facet': 'true',
    # What did Apache Tika think?
    'facet.field': 'content_type_tika',
    'facet.mincount' : 1
}

# Run the request:
r = requests.post(url='%s/select' % solr_endpoint, data=q, proxies=proxies)

# Report request time
print("Request took %s seconds." % r.elapsed.total_seconds())
#print(r.text)

# Parse the response:
results = json.loads(r.text)
tika_types = results['facet_counts']['facet_fields']['content_type_tika']

# Now we need to split the series of alternating (format, count) values:
tika_data = {
    'content_type_tika': tika_types[::2],
    'count': tika_types[1::2]
}

# And we can turn this into a proper table:
import pandas as pd

# Make sure we can see all the rows:
pd.set_option('display.max_rows', 500)
# Do not truncate entries:
pd.set_option('display.max_colwidth', -1)


# Show as a table:
pd.DataFrame(tika_data)

Request took 7.020886 seconds.


,content_type_tika,count
0,text/plain; charset=ISO-8859-1,31449260
1,application/javascript; charset=ISO-8859-1,28702415
2,text/css; charset=ISO-8859-1,20663757
3,text/css; charset=windows-1252,9717546
4,text/x-php; charset=ISO-8859-1,9046591
5,text/html; charset=ISO-8859-1,7925612
6,application/javascript; charset=windows-1252,6719082
7,text/plain; charset=windows-1252,5795133
8,application/octet-stream,4660277
9,application/javascript; charset=UTF-8,4585871


We can quickly see that the vast majority of the issues relate to the identification of text formats, which are known to be difficult to identify. Here, Apache Tika appears to do slightly better than DROID/Nanite, but we can't be sure how authoritative these results are, for a couple of reasons.

Firstly, Apache Tika uses a simple heuristic rule to classify a bitstream as `text/plain`, based on the absense of byte values outside of the usual range for text encodings. This is fairly reliable, but the actual text could be in any one of the many text-based formats that our tools cannot yet identify.

Secondly, Apache Tika is more forgiving that DROID/Nanite.  The PRONOM signatures tend to assume bitstreams are 'neater' than they often are on the web. Sometimes files have oddities or gaps that break the rules, but the files will work in the more forgiving environment of a user's web browser or desktop.  And this means some items in formats like JPEG, PNG or PDF sometimes slip past PRONOM's signatures.  But it's not clear that making PRONOM's signatures looser would be welcome in the contexts that DROID is usually deployed.

Finally, it's also notable that there are quite a few items that come up as unknown to Tika too.

Looking for new binary signatures
-------------------------------------------------

So, putting those difficulties and disagreements aside, we can still make some progress on the remaining unknown formats. This is because, with this use case in mind, the webarchive-discovery toolkit was built to store additional fields:

- The file extension, if any
- The MIME type reported by the original server, if any
- The first four bytes of each resource (encoded as a hexadecimal string).

These fields can be used to look for correlations between the first four bytes and the external identifiers, and generate candidate binary format signatures.

But to do this, we need to get Solr to dig a little deeper. In this case, we want to gather our unknown items and bring those with the same first four bytes together, and then break each of those down by the file extensions we found.

This should be possible using 'pivot' facet queries, but in practice I've found that these are slow and tend to stretch Solr too far, leading to failed queries.

However, we can perform more sophisticated analysis and data extraction using Solr's [Streaming Expressions](https://lucene.apache.org/solr/guide/6_6/streaming-expressions.html) support. This mode of interacting with queries allows large datasets to be streamed through and analysed and/or extracted.

It's still slow, but it's a bit more reliable.


In [69]:
q = {
    # Run the query over the nodes in the 'all' collection
    'expr': 'parallel(all, \
        rollup( \
            search(%s, q="content_type_droid:\"application/octet-stream\" AND content_type_tika:\"application/octet-stream\"",\
            fl="content_ffb,content_type_ext",\
            sort="content_ffb asc, content_type_ext asc",\
            qt="/export", partitionKeys="content_ffb") \
          , over="content_ffb, content_type_ext", \
          count(*) \
        ) \
        , workers="%i", sort="content_ffb asc")' % (solr_collection, 25)
}

r = requests.post(url='%s/stream' % solr_endpoint, data=q, proxies=proxies)


# Print the results so we can see what happened:
print("In %s seconds." % r.elapsed.total_seconds())
print(r.text[0:1000])
print(r.text[-1000:])


In 8.632658 seconds.
{"result-set":{"docs":[{"count(*)":1814,"content_type_ext":"NULL","content_ffb":"00000000"},{"count(*)":1,"content_type_ext":"001","content_ffb":"00000000"},{"count(*)":1,"content_type_ext":"1","content_ffb":"00000000"},{"count(*)":1,"content_type_ext":"100m","content_ffb":"00000000"},{"count(*)":6,"content_type_ext":"16","content_ffb":"00000000"},{"count(*)":2,"content_type_ext":"2","content_ffb":"00000000"},{"count(*)":2,"content_type_ext":"3","content_ffb":"00000000"},{"count(*)":2,"content_type_ext":"4","content_ffb":"00000000"},{"count(*)":1,"content_type_ext":"90","content_ffb":"00000000"},{"count(*)":5,"content_type_ext":"act","content_ffb":"00000000"},{"count(*)":2,"content_type_ext":"aps","content_ffb":"00000000"},{"count(*)":11,"content_type_ext":"ashx","content_ffb":"00000000"},{"count(*)":1,"content_type_ext":"asp","content_ffb":"00000000"},{"count(*)":1,"content_type_ext":"axd","content_ffb":"00000000"},{"count(*)":1,"content_type_ext":"bad","content_f

Well, that's the raw output -- it needs a bit of work to turn it into a proper breakdown.

And we'll include a decoded version of the hexadecimal first-four-bytes while we're reformatting the data...

In [70]:
import pandas as pd
import binascii
pd.set_option('display.max_rows', 500)

# Parse the results doc:
res=json.loads(r.text)['result-set']['docs']
#print(json.dumps(res, indent=4)[0:4000])

# Make a dataframe, but drop the last two lines which are just metadata:
df = pd.DataFrame(res[:-2])

# Add column with translated characters:
def translate_hex(row):
    #print(row.content_ffb)
    if( row.content_ffb != 'NULL'):
        return str(binascii.unhexlify(row.content_ffb))[1:]
    else:
        return None
    
df['content_ffb_txt'] = df.apply( lambda row: translate_hex(row), axis=1)

# And sort to bring the common problem to the top:
df[(df['content_ffb'] != 'NULL') & (df['count(*)'] > 100)].sort_values(by=['count(*)'], ascending=False)

,content_ffb,content_type_ext,count(*),content_ffb_txt
49543,de120495,mo,156839,'\xde\x12\x04\x95'
17165,4d000000,NULL,12146,'M\x00\x00\x00'
22024,70006f00,balancer5,8336,'p\x00o\x00'
14575,3c736372,NULL,7758,'<scr'
14522,3c627220,NULL,7702,'<br '
2617,05444956,div,6388,'\x05DIV'
3026,09080800,NULL,5601,'\t\x08\x08\x00'
17297,4d697661,mvc,4081,'Miva'
19670,613a343a,dat,3983,'a:4:'
16737,4b000000,gdf,3441,'K\x00\x00\x00'


Now we can see what the most popular combinations of first-four-bytes and file extension are, for all the files that appear to be completely unknown to both DROID/Nanite and Apache Tika.

We can now go back to the index to grab a sample of each...

In [93]:
# Make a function that will grab a sample for any given first-four-bytes:
def get_a_sample(ffb):
    # Set the query parameters...
    q = {
        # Get files starting with these bytes:
        'q': 'content_ffb:"%s"' % ffb,
        # Only retreive the fields we need:
        'fl': 'url,crawl_date,wayback_date,content_type_served',
        # Get a random sample of 100 records:
        'sort': 'random_2382 asc',
        'rows': '100',
        # In JSON:
        'wt': 'json',
        'indent': 'true'
    }

    # Run the request:
    r = requests.post(url='%s/select' % solr_endpoint, data=q, proxies=proxies)

    # Report request time
    print("Request took %s seconds." % r.elapsed.total_seconds())
    if r.status_code != 200:
        print(r.text)

    # Parse the response:
    results = json.loads(r.text)
    return pd.DataFrame(results['response']['docs'])

# Grab the 'mo' sample:
get_a_sample('de120495')

Request took 5.006331 seconds.


,content_type_served,crawl_date,url,wayback_date
0,NaN,2014-10-24T05:24:39Z,http://musclefitnesshealth.co.uk/wp-content/plugins/mappress-google-maps-for-wordpress/languages/mappress.mo,20141024052439
1,text/plain,2014-10-23T18:24:42Z,http://millantandoori.co.uk/wp-content/plugins/wp-e-commerce/wpsc-languages/wpsc-de_DE.mo,20141023182442
2,text/plain; charset=UTF-8,2014-06-27T21:10:05Z,http://connections-southwest.co.uk/wp-content/plugins/form-maker/languages/form_maker-lt_LT.mo,20140627211005
3,text/plain,2014-10-26T07:06:09Z,http://ramsgatecars.co.uk/wp-content/plugins/si-captcha-for-wordpress/languages/si-captcha-nb_NO.mo,20141026070609
4,text/plain,2014-11-25T22:33:02Z,http://www.etjames-sons.co.uk/wp-content/themes/alora/framework/plugins/post-types-order/lang/cpt-pt_BR.mo,20141125223302
5,application/octet-stream,2014-07-22T22:10:41Z,http://housingadvicelondon.co.uk/wp-content/plugins/comprehensive-google-map-plugin/languages/cgmp-it_IT.mo,20140722221041
6,text/plain,2014-07-03T05:07:36Z,http://carpetcleaningharlow.co.uk/wp-content/plugins/wordpress-seo/languages/wordpress-seo-it_IT.mo,20140703050736
7,text/plain,2014-09-10T22:51:12Z,http://remadeit.co.uk/wp-content/plugins/mappress-google-maps-for-wordpress/languages/mappress-ja.mo,20140910225112
8,text/plain,2014-10-20T07:24:02Z,http://jewellitz.co.uk/wordpress/wp-content/plugins/contact-form-7/languages/wpcf7-ar.mo,20141020072402
9,text/plain,2014-09-14T09:22:05Z,http://soapycauldrons.co.uk/wp-content/plugins/wp-e-commerce/wpsc-languages/wpsc-tr.mo,20140914092205


So, all these `mo` files appears to be language files for WordPress sites! As is often the case, searching for the hex-encoded bytes is also useful (e.g. [de120495](https://www.google.com/search?q=de120495) or [de120495 magic](https://www.google.com/search?q=de120495+magic)), and I can quickly find [this forum post](http://mark0.net/forum/index.php?topic=682.0) in the forums for the [TrID file identification tool](http://mark0.net/soft-trid-e.html) which has lots of additional details.

Looking the `mo` extension across multiple format registries, I can see TrID is currently the only source included there that knows this particular format: http://www.digipres.org/formats/extensions/#*.mo

The next most common value for the first four bytes is `4d000000`, and we can use the same approach to look at what's going on there.

In [94]:
get_a_sample('4d000000')

Request took 9.488989 seconds.


,content_type_served,crawl_date,url,wayback_date
0,text/csv; charset=utf-32,2018-11-06T06:05:30Z,https://hansard.parliament.uk/division/HOLDT20120618DIV1/downloadcsv?isEvel=False,20181106060530
1,text/csv; charset=utf-32,2016-10-08T21:58:13Z,https://hansard.parliament.uk/division/13062570001950/downloadcsv?isEvel=False,20161008215813
2,text/csv; charset=utf-32,2018-06-05T07:57:32Z,https://hansard.parliament.uk/division/120703128001314/downloadcsv?isEvel=False,20180605075732
3,text/csv; charset=utf-32,2018-11-07T13:49:39Z,https://hansard.parliament.uk/division/06071361000606/downloadcsv?isEvel=False,20181107134939
4,text/csv; charset=utf-32,2018-06-03T16:28:39Z,https://hansard.parliament.uk/division/14032588003538/downloadcsv?isEvel=False,20180603162839
5,text/csv; charset=utf-32,2018-08-11T01:03:04Z,https://hansard.parliament.uk/division/16012529000204/downloadcsv?isEvel=False,20180811010304
6,text/csv; charset=utf-32,2018-11-04T03:13:42Z,https://hansard.parliament.uk/division/15071446003277/downloadcsv?isEvel=False,20181104031342
7,text/csv; charset=utf-32,2016-10-08T18:18:59Z,https://hansard.parliament.uk/division/1210301000454/downloadcsv?isEvel=False,20161008181859
8,text/csv; charset=utf-32,2018-06-04T22:08:16Z,https://hansard.parliament.uk/division/12041873001811/downloadcsv?isEvel=False,20180604220816
9,text/csv; charset=utf-32,2018-11-03T06:59:11Z,https://hansard.parliament.uk/division/12032080001276/downloadcsv?isEvel=False,20181103065911


In this case, it looks like we can trust the content type sent by the original server, as these all appear to be 32-bit encoded Unicode text data files in CSV format, from Hansard.  While CSV is hard to spot, Apache Tika's text-detection heuristic appears to have failed to spot that this is plain text at all, due to the relatively unusual `UTF-32` encoding being used.

Searching for `70006f00` also appears to give CSV, but here Apache Tika appears to have missed the `UTF-16` encoded text:

In [95]:
get_a_sample('70006f00')

Request took 6.074177 seconds.


,content_type_served,crawl_date,url,wayback_date
0,text/csv,2013-05-17T17:20:06Z,http://www.charm.kcl.ac.uk/discography/search/Hardy-xls03.csv;jsessionid=3AEFFE91BF6D36E2689F20BB9E7A78CD.balancer5,20130517172006
1,text/csv,2013-04-16T12:53:10Z,http://www.charm.rhul.ac.uk/discography/search/Nicholls/Gilbert-xls07.csv;jsessionid=27E07ED6FE09F47DE0C8590D7897BAE9.balancer5,20130416125310
2,text/csv,2013-05-14T23:36:21Z,http://www.charm.kcl.ac.uk/discography/search/Oberthor-xls03.csv;jsessionid=96FF4458D4B86A67D683900314ECA13F.balancer5,20130514233621
3,text/csv,2013-05-13T03:40:42Z,http://www.charm.kcl.ac.uk/discography/search/E_B_Vald%C3%A8s/Petralia-xls07.csv;jsessionid=96FF4458D4B86A67D683900314ECA13F.balancer5,20130513034042
4,text/csv,2013-05-09T22:49:03Z,http://www.charm.kcl.ac.uk/discography/search/Zabel-xls07.csv;jsessionid=2950CC77762EB05DCBF37923E6D01107.balancer5,20130509224903
5,text/csv,2013-05-13T04:23:41Z,http://www.charm.kcl.ac.uk/discography/search/Gabaroche/Bos-xls03.csv;jsessionid=96FF4458D4B86A67D683900314ECA13F.balancer5,20130513042341
6,text/csv,2013-05-16T23:34:50Z,http://www.charm.kcl.ac.uk/discography/search/N_A_ROSTOVSKY-xls03.csv;jsessionid=96FF4458D4B86A67D683900314ECA13F.balancer5,20130516233450
7,text/csv,2013-05-13T04:38:10Z,http://www.charm.kcl.ac.uk/discography/search/F_G_Gim%C3%A9nez/J_C_Mateo-xls03.csv;jsessionid=96FF4458D4B86A67D683900314ECA13F.balancer5,20130513043810
8,text/csv,2013-05-13T02:44:30Z,http://www.charm.kcl.ac.uk/discography/search/Quintero/Chap%C3%AD-xls03.csv;jsessionid=96FF4458D4B86A67D683900314ECA13F.balancer5,20130513024430
9,text/csv,2013-05-17T17:48:41Z,http://www.charm.kcl.ac.uk/discography/search/Talcen-xls03.csv;jsessionid=3AEFFE91BF6D36E2689F20BB9E7A78CD.balancer5,20130517174841


A further example, `09080800`, appears to be CSV based on the URL name, but it's less clear what's going on. 

In [97]:
get_a_sample('09080800')

Request took 19.100326 seconds.


,content_type_served,crawl_date,url,wayback_date
0,application/octet-stream,2018-10-23T23:01:32Z,http://www.garstonladiesfc.co.uk/club/234222/Fixtures/viewCsv,20181023230132
1,application/octet-stream,2014-12-15T23:15:53Z,http://www.thealetigers.co.uk/club/119478/LeagueTable/viewCsv,20141215231553
2,application/vnd.ms-excel,2018-04-01T18:16:43Z,https://www.leaguewebsite.co.uk/teamersportsleague/Divisions/viewCsv?div=19009&age=22,20180401181643
3,application/octet-stream,2014-12-06T01:45:07Z,http://www.ribbletonfc.co.uk/club/243450/Fixtures/viewCsv,20141206014507
4,text/plain,2014-10-18T02:50:13Z,http://fq.dev.3dmw.co.uk/tmp/510a4bfe8b86f,20141018025013
5,application/octet-stream,2018-01-31T04:32:43Z,https://www.clubwebsite.co.uk/taffswellafc/Fixtures/viewCsv/prevresults/2013-01-05,20180131043243
6,application/octet-stream,2014-11-05T04:12:12Z,http://www.garstonladiesfc.co.uk/club/114587/ResultsGrid/viewCsv,20141105041212
7,application/octet-stream,2014-11-04T23:23:19Z,http://www.fulbournfalcons.co.uk/club/280210/Fixtures/viewCsv,20141104232319
8,application/octet-stream,2018-04-01T06:48:13Z,https://www.clubwebsite.co.uk/rhayadertown/287640/ResultsGrid/viewCsv,20180401064813
9,application/octet-stream,2014-12-13T07:37:30Z,http://www.lkgalaxyfc.co.uk/club/286482/ResultsGrid/viewCsv,20141213073730


Taking one example and running some identification tools on it gets `Applesoft BASIC program data` (from `file`) or `(.XLS) Microsoft Excel worksheet (generic older format)` from TrID. The latter seems more likely!

A clearer example some from these `div` files, all starting with `05444956`:

In [96]:
get_a_sample('05444956')

Request took 2.439159 seconds.


,content_type_served,crawl_date,url,wayback_date
0,text/plain,2014-11-16T03:17:55Z,http://ltrackfiles.co.uk/files/ned-2013-1.div,20141116031755
1,text/plain,2013-05-10T02:43:57Z,http://www.ltrackfiles.co.uk/files/eng-1935-2.div,20130510024357
2,text/plain,2014-11-16T02:47:04Z,http://ltrackfiles.co.uk/files/ger-1988-2.div,20141116024704
3,text/plain,2014-08-06T03:42:15Z,http://ltrackfiles.co.uk/files/eng-2008-s5-uc1.div,20140806034215
4,text/plain,2013-04-23T04:56:24Z,http://www.ltrackfiles.co.uk/files/crc-2011-1v.div,20130423045624
5,text/plain,2013-05-10T02:56:58Z,http://www.ltrackfiles.co.uk/files/eng-2001-3.div,20130510025658
6,text/plain,2013-05-10T02:10:08Z,http://www.ltrackfiles.co.uk/files/nir-2011-w1.div,20130510021008
7,text/plain,2013-04-23T05:05:48Z,http://www.ltrackfiles.co.uk/files/eng-2011-s5-es1.div,20130423050548
8,text/plain,2014-08-06T00:13:13Z,http://ltrackfiles.co.uk/files/eng-1958-3s.div,20140806001313
9,text/plain,2014-11-16T03:06:14Z,http://ltrackfiles.co.uk/files/ita-1979-1.div,20141116030614


Visiting http://www.ltrackfiles.co.uk/, we find _'This site contains many data files for use with the free football statistics tracker Ltrack'_, so this appears to be a niche format used to track football statistics. There doesn't appear to be a format specification immediately available, but we have a reasonable starting point for a signature nevertheless.

Next, looking at `4d697661`...

In [98]:
get_a_sample('4d697661')

Request took 4.287166 seconds.


,content_type_served,crawl_date,url,wayback_date
0,text/plain,2014-06-27T04:03:33Z,http://chemken.co.uk/test/miva/test.mvc,20140627040333
1,text/plain,2014-10-29T10:26:14Z,http://taor.org.uk/test/miva/test.mvc,20141029102614
2,text/plain,2013-04-18T03:38:20Z,http://www.bswalouette.co.uk/test/miva/test.mvc,20130418033820
3,text/plain,2014-06-27T08:02:47Z,http://civilweddingvenue.co.uk/test/miva/test.mvc,20140627080247
4,text/plain,2013-04-15T00:38:40Z,http://images.kitchens-review.co.uk/test/miva/test.mvc,20130415003840
5,text/plain,2014-08-16T00:37:31Z,http://mweconsulting.co.uk/test/miva/test.mvc,20140816003731
6,text/plain,2014-09-16T04:56:02Z,http://spreeweb.co.uk/test/miva/test.mvc,20140916045602
7,text/plain,2014-06-23T23:30:38Z,http://belatedbirthdaycard.co.uk/mm5/5.00/modules/component/cmp-mmui-invc-custfields.mvc,20140623233038
8,text/plain,2014-10-26T05:05:51Z,http://ppcaccounts.co.uk/test/miva/test.mvc,20141026050551
9,text/plain,2014-10-27T17:20:55Z,http://shadeshut.co.uk/test/miva/test.mvc,20141027172055


These all appear to be test files associated with the [MIVA Script](https://en.wikipedia.org/wiki/MIVA_Script) programming language, in some compiled form.

There's also a lot of `mso` files, like these:

In [100]:
get_a_sample('41637469')

Request took 3.336974 seconds.


,content_type_served,crawl_date,url,wayback_date
0,text/plain,2013-07-08T16:39:29Z,http://www.chesterfieldroyal.nhs.uk/dl/9501_5267115678.txt/as/Activites%20to%20develop%20early%20play%20skills%20-%20version%201%20-%20Oct%202014.txt?_ts=2004212&_ts=2004212,20130708163929
1,text/html,2014-10-21T05:42:12Z,http://cdn2.peterharrington.co.uk/rare-books/?orderby=date&order=desc&mvn-insert-to-cart=286563&mvn-qty=1&_wpnonce=27b275a30e,20141021054212
2,text/plain,2013-04-19T13:39:48Z,http://www.legoville.co.uk/6_files/editdata.mso,20130419133948
3,application/octet-stream,2013-06-06T22:29:59Z,http://www1.somerset.gov.uk/council/board3b/2006%20March%208%20Item%205%20Northmoor%20Green%20Primary%20School_files/editdata.mso,20130606222959
4,text/html,2014-07-11T19:02:40Z,http://francoisconsulting.co.uk/eCommercenew_files/editdata.mso,20140711190240
5,text/plain,2013-05-04T09:10:38Z,http://istug.fast.net.uk/docs/June03mtg/session5_files/editdata.mso,20130504091038
6,text/plain,2014-11-20T21:16:06Z,http://www.dawlishweather.co.uk/history/2006/holcombe_dec_06_sum_files/editdata.mso,20141120211606
7,text/plain,2013-04-17T22:05:52Z,http://www.construct-it.org.uk/pages/events/members_meetings/November_2003/presentations/Jarmo%20Laitinen%20-%20JT%20Innovators%20Ltd_files/editdata.mso,20130417220552
8,text/plain,2013-04-19T12:41:27Z,http://www.legoville.co.uk/95_files/editdata.mso,20130419124127
9,application/octet-stream,2018-06-12T17:51:02Z,https://www.iwight.com/council/committees/Scrutiny%20Committee/18-1-07/Minutes_files/editdata.mso,20180612175102


Going via [digipres.org][http://www.digipres.org/formats/extensions/#*.mso], these all appear to be [ActiveMime](http://fileformats.archiveteam.org/wiki/ActiveMime) generated by Microsoft Office.

Finally, the `ipx` files make for an interesting example of obsolete files. We see two different binary signatures for these:

In [101]:
get_a_sample('7362cc01')

Request took 2.167514 seconds.


,content_type_served,crawl_date,url,wayback_date
0,application/x-ipx,2014-06-25T06:08:03Z,http://bournecoast-lettings.co.uk/bournecoastsync/IPIX_Lettings/Southbourne_Overcliff_Drive_92_Master_Bedroom.ipx,20140625060803
1,text/plain,2014-11-21T06:19:56Z,http://www.the360degrees.co.uk/namibia2/html/dinning.ipx,20141121061956
2,application/x-ipx,2014-11-21T10:16:46Z,http://www.bournecoast.co.uk/bournecoastsync/IPIX_Lettings/Saffron_Way_38_Bedroom2.ipx,20141121101646
3,text/plain,2014-11-22T06:16:29Z,http://www.uklink.co.uk/research/chatterboxnottingham/nottingclient.hires.ipx,20141122061629
4,application/octet-stream,2013-04-25T20:48:45Z,http://www.bw-falmouthbeachhotel.co.uk/ipix/falmouthbeach_pool_medres.ipx,20130425204845
5,NaN,2016-02-25T16:55:13Z,http://www.newlanark.org/ipix/no8bedroom.ipx,20160225165513
6,text/plain,2014-10-18T11:43:51Z,http://harthill-village.co.uk/ipix/3_hires.ipx,20141018114351
7,application/x-ipx,2014-06-24T23:27:14Z,http://bournecoast.co.uk/bournecoastsync/IPIX_Lettings/Seabourne_Road_117A_Living_Room.ipx,20140624232714
8,application/x-ipx,2014-06-25T05:37:52Z,http://bournecoast-lettings.co.uk/bournecoastsync/IPIX_Lettings/SOCD_-_111_Master_Bedroom.ipx,20140625053752
9,text/plain,2013-04-24T23:17:20Z,http://www.360spin.co.uk/education-government/fashion.ipx,20130424231720


In [102]:
get_a_sample('7362eb01')

Request took 3.772251 seconds.


,content_type_served,crawl_date,url,wayback_date
0,application/x-ipix,2013-04-14T21:39:49Z,http://lincswoldscottage.co.uk/bed2ap.ipx,20130414213949
1,text/plain,2013-04-24T22:10:46Z,http://www.360spin.co.uk/virtual-liverpool/cathedral.ipx,20130424221046
2,application/x-ipix,2013-04-17T13:00:19Z,http://www.craiglealodge.co.uk/v1/360/flat1_Living_med.ipx,20130417130019
3,application/octet-stream,2014-11-23T05:19:37Z,http://www.picklefarm.co.uk/ipix/bedroom1_ipix_med.ipx,20141123051937
4,application/x-ipix,2014-11-17T23:12:32Z,http://newenglandsuites.co.uk/VTour/quad2.ipx,20141117231232
5,text/plain,2014-10-14T19:13:43Z,http://360spin.co.uk/langtrymanor/LangtryLounge_ipix_med.ipx,20141014191343
6,text/plain,2013-04-25T00:15:05Z,http://www.360spin.co.uk/london/collegehall/Reception_ipix_med.ipx,20130425001505
7,text/plain,2013-04-24T23:41:56Z,http://www.360spin.co.uk/castlebankhotel/room2_ipix_med.ipx,20130424234156
8,text/plain,2013-04-24T23:42:28Z,http://www.360spin.co.uk/3sixtymedia-2/StarlightTheatre_ipix_med.ipx,20130424234228
9,text/plain,2013-04-24T23:35:38Z,http://www.360spin.co.uk/craiglea/SwimmingPool_med.ipx,20130424233538


These are two different versions of the proprietary [IPIX](https://en.wikipedia.org/wiki/IPIX) format, which was used to provide immersive 'VR-style' panoramas of internal and external locations. These are potentially fascinating windows into the past, but they have become very difficult to accessible due to relying on a custom Java applet for playback.

Summary
--------------

I hope this notebook provides some ideas as to how web archives can be used to explore rare formats, and to generate initial candidate binary signatures. Text-based formats remain a challenge for all tools, but combining Apache Tika with DROID/Nanite gives decent results for common web formats.